### data load

In [2]:
import sys
sys.path.append('..')

In [3]:
import json
from tqdm import tqdm
import random

import torch
import numpy as np
from transformers import BertTokenizer
from data_utils import get_examples_from_dialogues, convert_state_dict, load_dataset
from data_utils import OntologyDSTFeature, DSTPreprocessor, _truncate_seq_pair

In [39]:
train_dials = json.load(open('../../input/data/train_dataset/train_dials.json'))
slot_meta = json.load(open('../../input/data/train_dataset/slot_meta.json'))
ontology = json.load(open('../../input/data/train_dataset/ontology_aug.json'))

eval_dials = json.load(open('../../input/data/eval_dataset/eval_dials.json'))
eval_meta = json.load(open('../../input/data/eval_dataset/slot_meta.json'))

### train_dials

In [3]:
print("len_train_dials:", len(train_dials))
print("train_dials[0].keys(): ", train_dials[0].keys())

len_train_dials: 7000
train_dials[0].keys():  dict_keys(['dialogue_idx', 'domains', 'dialogue'])


### slot_meta

In [21]:
print(type(slot_meta))
slot_meta[:5]

<class 'list'>


['관광-경치 좋은', '관광-교육적', '관광-도보 가능', '관광-문화 예술', '관광-역사적']

### ontology-aug

In [32]:
print(type(ontology))
print(len(ontology))
print(ontology.keys())
len(ontology['식당-예약 시간'])  # 원래 길이 569
# ontology
arr = set()
for i in range(24):
    for j in range(60):
        arr.add(f'{str(i).zfill(2)}:{str(j).zfill(2)}')
arr.add('none')
arr.add('dontcare')
ontology['식당-예약 시간'] = list(arr)
len(ontology['식당-예약 시간'])

<class 'dict'>
45
dict_keys(['관광-경치 좋은', '관광-교육적', '관광-도보 가능', '관광-문화 예술', '관광-역사적', '관광-이름', '관광-종류', '관광-주차 가능', '관광-지역', '숙소-가격대', '숙소-도보 가능', '숙소-수영장 유무', '숙소-스파 유무', '숙소-예약 기간', '숙소-예약 명수', '숙소-예약 요일', '숙소-이름', '숙소-인터넷 가능', '숙소-조식 가능', '숙소-종류', '숙소-주차 가능', '숙소-지역', '숙소-헬스장 유무', '숙소-흡연 가능', '식당-가격대', '식당-도보 가능', '식당-야외석 유무', '식당-예약 명수', '식당-예약 시간', '식당-예약 요일', '식당-이름', '식당-인터넷 가능', '식당-종류', '식당-주류 판매', '식당-주차 가능', '식당-지역', '식당-흡연 가능', '지하철-도착지', '지하철-출발 시간', '지하철-출발지', '택시-도착 시간', '택시-도착지', '택시-종류', '택시-출발 시간', '택시-출발지'])


1442

In [37]:
ontology['택시-출발 시간'] = list(arr)
sorted(ontology['택시-출발 시간'])

ontology['택시-도착 시간'] = list(arr)
sorted(ontology['택시-도착 시간'])[-10:]

ontology['지하철-출발 시간'] = list(arr)
sorted(ontology['지하철-출발 시간'])[-10:]

['23:52',
 '23:53',
 '23:54',
 '23:55',
 '23:56',
 '23:57',
 '23:58',
 '23:59',
 'dontcare',
 'none']

In [67]:
seoul_subway = json.load(open('../../input/data/train_dataset/seoul_subway.json'))
arr =[f"{e['station_nm']}역" for e in seoul_subway['DATA']]
arr = list(set(arr))
arr.remove('서울역역')
arr.append('서울역')
arr
ontology['지하철-출발지'].extend(arr)
ontology['지하철-출발지'] = sorted(list(set(ontology['지하철-출발지'])))
print(ontology['지하철-출발지'])
len(ontology['지하철-출발지'])

['dontcare', 'none', '가락시장역', '가산디지털단지역', '강남구청역', '강남역', '강동구청역', '강동역', '강변역', '강일역', '개롱역', '개화산역', '거여역', '건대입구역', '경복궁역', '경찰병원역', '고덕역', '고려대역', '고속터미널역', '공덕역', '공릉역', '광나루역', '광명사거리역', '광화문역', '광흥창역', '교대역', '구로디지털단지역', '구산역', '구의역', '구파발역', '국회의사당역', '군자역', '굴포천역', '굽은다리역', '금호역', '길동역', '길음역', '김포공항역', '까치산역', '까치울역', '낙성대역', '남구로역', '남부터미널역', '남성역', '남태령역', '남한산성입구역', '내방역', '노량진역', '노원역', '녹번역', '녹사평역', '논현역', '단대오거리역', '답십리역', '당고개역', '당산역', '대림역', '대청역', '대치역', '대흥역', '도곡역', '도림천역', '도봉산역', '독립문역', '독바위역', '돌곶이역', '동대문역', '동대문역사문화공원역', '동대입구역', '동묘앞역', '동작역', '둔촌동역', '둔촌오륜역', '디지털미디어시티역', '뚝섬역', '뚝섬유원지역', '마곡역', '마들역', '마장역', '마천역', '마포구청역', '마포역', '망원역', '매봉역', '먹골역', '면목역', '명동역', '명일역', '모란역', '목동역', '몽촌토성역', '무악재역', '문래역', '문정역', '미사역', '미아사거리역', '미아역', '반포역', '발산역', '방배역', '방이역', '방화역', '버티고개역', '보라매역', '보문역', '복정역', '봉은사역', '봉천역', '봉화산역', '부천시청역', '부천종합운동장역', '부평구청역', '불광역', '사가정역', '사당역', '산성역', '삼각지역', '삼산체육관역', '삼성역', '삼성중앙역', '삼전역', '상계역', '상도역', '상동역', '상봉역', '

266

In [69]:
ontology['지하철-도착지'].extend(arr)
ontology['지하철-도착지'] = sorted(list(set(ontology['지하철-도착지'])))
len(ontology['지하철-도착지'])

266

In [70]:
with open('ontology_aug.json', 'w') as outfile:
    json.dump(ontology, outfile)